In [ ]:
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import display
import matplotlib.pyplot as plt
import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings; warnings.filterwarnings('ignore')
import cv2
import pydicom
#from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score

from pylab import rcParams
rcParams['figure.figsize'] = 20,5

In [ ]:
!gdown --id 1diztKyKFfhINpY-8R_KB84kV8akW2NjD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1diztKyKFfhINpY-8R_KB84kV8akW2NjD
To: /content/test_train_val.zip
100% 480M/480M [00:01<00:00, 280MB/s]


In [ ]:
!unzip -qq test_train_val.zip

replace test_train_val/test_meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
with open('test_train_val/train_meta.csv') as f:
  train_df= pd.read_csv(f)
with open('test_train_val/train_signal.csv') as f:
  train_signal = pd.read_csv(f)

with open('test_train_val/valid_meta.csv') as f:
  valid_df= pd.read_csv(f)
with open('test_train_val/valid_signal.csv') as f:
  valid_signal = pd.read_csv(f)

with open('test_train_val/test_meta.csv') as f:
  test_df= pd.read_csv(f)
with open('test_train_val/test_signal.csv') as f:
  test_signal = pd.read_csv(f)

print(train_df.shape)

train_df

(17441, 37)


,ecg_id,age,sex,height,weight,nurse,site,device,NORM,MI,...,sub_ISC_,sub_SEHYP,sub_ISCI,sub_CRBBB,sub_CLBBB,sub_LAO/LAE,sub_ILBBB,sub_WPW,sub_PMI,strat_fold
0,1,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,3
1,2,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,2
2,3,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,5
3,4,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,3
4,5,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17436,21832,63.0,0,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,7
17437,21833,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,7
17438,21834,93.0,0,NaN,NaN,1.0,2.0,AT-60 3,1,0,...,0,0,0,0,0,0,0,0,0,4
17439,21835,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
# train_df=pd.read_csv("C:/Users/Abdulhaq/Documents/test_train_val/test_train_val/train_meta.csv")
# train_signal=pd.read_csv("C:/Users/Abdulhaq/Documents/test_train_val/test_train_val/train_signal.csv")


# valid_df=pd.read_csv("C:/Users/Abdulhaq/Documents/test_train_val/test_train_val/valid_meta.csv")
# valid_signal=pd.read_csv("C:/Users/Abdulhaq/Documents/test_train_val/test_train_val/valid_signal.csv")


# test_df=pd.read_csv("C:/Users/Abdulhaq/Documents/test_train_val/test_train_val/test_meta.csv")
# test_signal=pd.read_csv("C:/Users/Abdulhaq/Documents/test_train_val/test_train_val/test_signal.csv")


# print(train_df.shape)

# train_df

In [ ]:
class PTBXLDatasetPreprocesser():
    def __init__(self):
        pass
    
    def save(self, filename):
        data = {
            'superclass_cols': self.superclass_cols,
            'subclass_cols': self.subclass_cols,
            'meta_num_cols': self.meta_num_cols,
            'meta_num_means': self.meta_num_means,
            'min_max_scaler': self.min_max_scaler,
            'meta_cat_cols': self.meta_cat_cols,
            'cat_lablers': self.cat_lablers,
            
        }
        pd.to_pickle(data, filename)
        
    def load(self, filename):
        data = pd.read_pickle(filename)
        self.min_max_scaler = data['min_max_scaler']
        self.cat_lablers = data['cat_lablers']
        
        
    def fit(self, x, y):
        x = x.copy()
        y = y.copy()
        
        self.superclass_cols = ['NORM', 'MI', 'STTC', 'CD', 'HYP']
        
        self.subclass_cols = [col for col in y.columns if 'sub_' in col]

        
        
        self.meta_num_cols = ['age', 'height', 'weight']
        self.meta_num_means = []
        for col in self.meta_num_cols:
            print(col, y[col].mean())
            y[col] = y[col].fillna(y[col].mean())
            self.meta_num_means += [y[col].mean()]
            
        self.min_max_scaler = MinMaxScaler().fit(y[self.meta_num_cols])
        
        self.meta_cat_cols = ['sex'] #, 'nurse', 'device']
        self.cat_lablers = [LabelEncoder().fit(y[col].fillna('none').astype(str)) for col in self.meta_cat_cols]
        return self

        
    def transform(self, x, y):
        
        channel_cols = x.columns.tolist()[1:]
        
        ret = []
        x = x[channel_cols].values.reshape(-1, 1000, 12)
        print(x.shape)
        ret += [x] # signal
        
        y_ = y.copy()
        
        for i, col in enumerate(self.meta_num_cols):
            y_[col] = y_[col].fillna(self.meta_num_means[i])
        y_[self.meta_num_cols] = self.min_max_scaler.transform(y_[self.meta_num_cols])
        y_[self.meta_num_cols] = np.clip(y_[self.meta_num_cols], 0., 1.) # prevent extreme value far from train set
        
        ret += [y_[self.meta_num_cols]] # meta num features
        
        for i, col in enumerate(self.meta_cat_cols):
            y_[col] = y_[col].fillna('none').astype(str)
            y_[col] = self.cat_lablers[i].transform(y_[col]) 
        
        ret += [y_[self.meta_cat_cols]] # meta cat features

        
        
        if np.isin(self.superclass_cols, y.columns).sum() == len(self.superclass_cols):
            ret += [y[self.superclass_cols].fillna(0).astype(int)] # superclass targets
        
        if np.isin(self.subclass_cols, y.columns).sum() == len(self.subclass_cols):
            ret += [y[self.subclass_cols].fillna(0).astype(int)] # subclass targets

        
        return ret

In [ ]:
data_preprocessor = PTBXLDatasetPreprocesser()
data_preprocessor.fit(train_signal, train_df)
train_signal, train_meta_num_feats, train_meta_cat_feats, train_superclass, train_subclass = data_preprocessor.transform(train_signal, train_df)
valid_signal, valid_meta_num_feats, valid_meta_cat_feats, valid_superclass, valid_subclass = data_preprocessor.transform(valid_signal, valid_df)
test_signal, test_meta_num_feats, test_meta_cat_feats, test_superclass, test_subclass = data_preprocessor.transform(test_signal, test_df)

print(train_signal.shape)
print(valid_meta_num_feats.isna().sum(), valid_meta_cat_feats.isna().sum(), valid_superclass.isna().sum(), valid_subclass.isna().sum())

display(train_meta_num_feats)
display(train_meta_cat_feats)
display(train_superclass)
display(train_subclass)

age 59.63709074169975
height 166.68908469699693
weight 70.69599447513812
(17441, 1000, 12)
(2193, 1000, 12)
(2203, 1000, 12)
(17441, 1000, 12)
age       0
height    0
weight    0
dtype: int64 sex    0
dtype: int64 NORM    0
MI      0
STTC    0
CD      0
HYP     0
dtype: int64 sub_NORM         0
sub_IMI          0
sub_STTC         0
sub_NST_         0
sub_LVH          0
sub_LAFB/LPFB    0
sub_RVH          0
sub_RAO/RAE      0
sub_IRBBB        0
sub_IVCD         0
sub_LMI          0
sub_AMI          0
sub__AVB         0
sub_ISCA         0
sub_ISC_         0
sub_SEHYP        0
sub_ISCI         0
sub_CRBBB        0
sub_CLBBB        0
sub_LAO/LAE      0
sub_ILBBB        0
sub_WPW          0
sub_PMI          0
dtype: int64


,age,height,weight
0,0.580645,0.791572,0.282927
1,0.182796,0.791572,0.317073
2,0.376344,0.791572,0.312195
3,0.236559,0.791572,0.375610
4,0.182796,0.791572,0.317073
...,...,...,...
17436,0.655914,0.791572,0.320468
17437,0.698925,0.791572,0.320468
17438,0.978495,0.791572,0.320468
17439,0.612903,0.791572,0.320468


,sex
0,1
1,0
2,1
3,0
4,1
...,...
17436,0
17437,1
17438,0
17439,1


,NORM,MI,STTC,CD,HYP
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
17436,0,0,0,1,0
17437,0,0,1,0,0
17438,1,0,0,0,0
17439,0,0,1,0,0


,sub_NORM,sub_IMI,sub_STTC,sub_NST_,sub_LVH,sub_LAFB/LPFB,sub_RVH,sub_RAO/RAE,sub_IRBBB,sub_IVCD,...,sub_ISCA,sub_ISC_,sub_SEHYP,sub_ISCI,sub_CRBBB,sub_CLBBB,sub_LAO/LAE,sub_ILBBB,sub_WPW,sub_PMI
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17436,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
17437,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17438,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17439,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
signal=np.array(train_signal)

In [ ]:
target=np.array(train_superclass)
target.shape

(17441, 5)

In [ ]:
v_signal=np.array(valid_signal)

In [ ]:
v_target=np.array(valid_superclass)
v_target.shape

(2193, 5)

In [ ]:
t_signal=np.array(test_signal)

In [ ]:
t_target=np.array(test_superclass)

In [ ]:
full_signal= np.vstack((signal,v_signal,t_signal))
full_signal.shape

(21837, 1000, 12)

In [ ]:
full_target= np.vstack((target,v_target,t_target))
full_target.shape

(21837, 5)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(full_signal, full_target, test_size=0.15, random_state=25)

In [ ]:
print("train signal shape", X_train.shape,"train target shape",Y_train.shape)
print("test signal shape", X_test.shape,"test target shape",Y_test.shape)

train signal shape (18561, 1000, 12) train target shape (18561, 5)
test signal shape (3276, 1000, 12) test target shape (3276, 5)


In [ ]:
!pip install ecg-plot
#import physionet_challenge_utility_script as pc
import ecg_plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
#from keras.utils import plot_model
#from keras.preprocessing.sequence import pad_sequences
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPool1D, ZeroPadding1D, LSTM, Bidirectional
from keras.models import Sequential, Model
#from keras.utils import plot_model
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, auc, precision_score, recall_score, f1_score
from sklearn import metrics


from keras.layers.merge import concatenate
from scipy import optimize
from scipy.io import loadmat
import os
%load_ext autoreload
%autoreload
%reload_ext autoreload

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.1, patience=3, verbose=1, mode='max',
    min_delta=0.0001, cooldown=0, min_lr=0
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10)

**5 MultiClass Classification**

**ANN Model**

In [ ]:
ann_model = Sequential()
ann_model.add(Dense(50, activation='relu', input_shape=(1000,12)))
ann_model.add(Dense(50, activation='relu'))
ann_model.add(Dense(50, activation='relu'))
ann_model.add(Dense(50, activation='relu'))
ann_model.add(GlobalAveragePooling1D())
ann_model.add(Dense(5, activation='softmax'))

In [ ]:
# ann_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
ann_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
ann_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000, 50)          650       
                                                                 
 dense_1 (Dense)             (None, 1000, 50)          2550      
                                                                 
 dense_2 (Dense)             (None, 1000, 50)          2550      
                                                                 
 dense_3 (Dense)             (None, 1000, 50)          2550      
                                                                 
 global_average_pooling1d (G  (None, 50)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_4 (Dense)             (None, 5)                 255       
                                                        

In [ ]:
# plot_model(ann_model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = ann_model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = ann_model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred1=ann_model.predict(X_test)
  predict1=np.argmax(y_pred1, axis=1)
  y_test1=np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test1, predict1)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test1, predict1,average='micro')
  rec_score = recall_score(y_test1, predict1,average='micro')
  f_score = f1_score(y_test1, predict1,average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)

Training on Fold:  1
Epoch 1/30
493/493 [==============================] - 6s 6ms/step - loss: 0.5089 - accuracy: 0.4760 - val_loss: 0.4650 - val_accuracy: 0.5576 - lr: 0.0010
Epoch 2/30
493/493 [==============================] - 2s 5ms/step - loss: 0.4399 - accuracy: 0.5687 - val_loss: 0.4347 - val_accuracy: 0.5899 - lr: 0.0010
Epoch 3/30
493/493 [==============================] - 3s 6ms/step - loss: 0.4140 - accuracy: 0.5934 - val_loss: 0.4163 - val_accuracy: 0.6065 - lr: 0.0010
Epoch 4/30
493/493 [==============================] - 3s 7ms/step - loss: 0.4018 - accuracy: 0.6086 - val_loss: 0.4075 - val_accuracy: 0.6004 - lr: 0.0010
Epoch 5/30
493/493 [==============================] - 4s 8ms/step - loss: 0.3953 - accuracy: 0.6141 - val_loss: 0.4047 - val_accuracy: 0.6237 - lr: 0.0010
Epoch 6/30
493/493 [==============================] - 4s 7ms/step - loss: 0.3909 - accuracy: 0.6182 - val_loss: 0.4050 - val_accuracy: 0.6158 - lr: 0.0010
Epoch 7/30
493/493 [=============================

ValueError: ignored

In [ ]:
# batchsize = 32
# ann_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop], validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 10s 11ms/step - loss: 0.4945 - accuracy: 0.5154 - val_loss: 0.4460 - val_accuracy: 0.5723 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 7s 13ms/step - loss: 0.4269 - accuracy: 0.5809 - val_loss: 0.4154 - val_accuracy: 0.5905 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 7s 13ms/step - loss: 0.4051 - accuracy: 0.6009 - val_loss: 0.4231 - val_accuracy: 0.5828 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 7s 13ms/step - loss: 0.3973 - accuracy: 0.6094 - val_loss: 0.4046 - val_accuracy: 0.5850 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 4s 8ms/step - loss: 0.3906 - accuracy: 0.6160 - val_loss: 0.4108 - val_accuracy: 0.5955 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 4s 7ms/step - loss: 0.3863 - accuracy: 0.6226 - val_loss: 0.3962 - val_accuracy: 0.6161 - lr: 0.0010
Epoch 7/50
546/546 [==============================] - 4s 7ms/step

In [ ]:
# plt.plot(ann_model.history.history['accuracy'])
# # plt.plot(ann_model.history.history['Precision'])
# plt.plot(ann_model.history.history['val_accuracy'])
# # plt.plot(ann_model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(ann_model.history.history['loss'])
# plt.plot(ann_model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# plt.plot(ann_model.history.history['Recall'])
# plt.plot(ann_model.history.history['Precision'])
# plt.plot(ann_model.history.history['AUC'])
# plt.legend(("Recall","Precision","AUC"))
# plt.xlabel('Epoch')

In [ ]:
# y_pred=ann_model.predict(signal_test)
# # predict=[int(x+0.5) for x in y_pred]
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test, axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# #conf_mat=confusion_matrix((target_test,predict))
# print(conf_mat)
# # print(y_pred)

[[876  36  39  50   3]
 [142 195 109  90  17]
 [ 73  52 207  30  17]
 [ 72  26   6  99   8]
 [ 33   1   7   5  10]]


In [ ]:
# accuracy_score(y_test,y_pred)

0.6295960054471176

**CNN Models**

**lenet-5 Model**

In [ ]:
lenet_5_model=Sequential()

lenet_5_model.add(Conv1D(filters=6, kernel_size=3, padding='same', input_shape=(1000,12)))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

lenet_5_model.add(Conv1D(filters=16, strides=1, kernel_size=5))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

lenet_5_model.add(GlobalAveragePooling1D())

lenet_5_model.add(Dense(64, activation='relu'))

lenet_5_model.add(Dense(32, activation='relu'))

lenet_5_model.add(Dense(5, activation = 'sigmoid'))

In [ ]:
# lenet_5_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
lenet_5_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
lenet_5_model.summary()

In [ ]:
# plot_model(lenet_5_model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = lenet_5_model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = lenet_5_model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred2= lenet_5_model.predict(X_test)
  predict1=np.argmax(y_pred2, axis=1)
  y_test2=np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test2, predict2)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test2, predict2,average='micro')
  rec_score = recall_score(y_test2, predict2,average='micro')
  f_score = f1_score(y_test2, predict2,average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)


In [ ]:
# batchsize = 32
# lenet_5_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop], validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 14s 7ms/step - loss: 0.4653 - accuracy: 0.5309 - val_loss: 0.4866 - val_accuracy: 0.5244 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 5s 8ms/step - loss: 0.3874 - accuracy: 0.6114 - val_loss: 0.4070 - val_accuracy: 0.6051 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 3s 6ms/step - loss: 0.3696 - accuracy: 0.6271 - val_loss: 0.3766 - val_accuracy: 0.6252 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 3s 6ms/step - loss: 0.3618 - accuracy: 0.6320 - val_loss: 0.3548 - val_accuracy: 0.6393 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 5s 9ms/step - loss: 0.3530 - accuracy: 0.6463 - val_loss: 0.3525 - val_accuracy: 0.6420 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 3s 6ms/step - loss: 0.3467 - accuracy: 0.6504 - val_loss: 0.3478 - val_accuracy: 0.6420 - lr: 0.0010
Epoch 7/50
546/546 [==============================] - 4s 6ms/step - l

In [ ]:
# plt.plot(lenet_5_model.history.history['accuracy'])
# #plt.plot(lenet_5_model.history.history['Precision'])
# plt.plot(lenet_5_model.history.history['val_accuracy'])
# #plt.plot(lenet_5_model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(lenet_5_model.history.history['loss'])
# plt.plot(lenet_5_model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# plt.plot(lenet_5_model.history.history['Recall'])
# plt.plot(lenet_5_model.history.history['Precision'])
# plt.plot(lenet_5_model.history.history['AUC'])
# plt.legend(("Recall","Precision","AUC"))
# plt.xlabel('Epoch')

In [ ]:
# y_pred=lenet_5_model.predict(signal_test)
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test, axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# print(conf_mat)

[[893  37  36  36   2]
 [111 213  96 128   5]
 [ 87  33 220  31   8]
 [ 63  10   5 130   3]
 [ 33   2  10   6   5]]


In [ ]:
# accuracy_score(y_test,y_pred)

0.6631865637766682

**AlexNet Model**

In [ ]:
alexNet_model=Sequential()

alexNet_model.add(Conv1D(filters=96, kernel_size=11, strides=4, input_shape=(1000,12)))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(Conv1D(filters=256, kernel_size=5, padding='same'))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(Conv1D(filters=384, padding='same', kernel_size=3))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(Conv1D(filters=384, kernel_size=3))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(Conv1D(filters=256, kernel_size=3))
alexNet_model.add(BatchNormalization())
alexNet_model.add(Activation('relu'))
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(GlobalAveragePooling1D())
alexNet_model.add(Dense(128, activation='relu'))
alexNet_model.add(Dropout(0.4))
alexNet_model.add(Dense(128, activation='relu'))
alexNet_model.add(Dropout(0.4))
alexNet_model.add(Dense(5, activation='sigmoid'))

In [ ]:
# alexNet_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
alexNet_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
alexNet_model.summary()

In [ ]:
# plot_model(alexNet_model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = alexNet_model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = alexNet_model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred3=alexNet_model.predict(X_test)
  predict3=np.argmax(y_pred3, axis=1)
  y_test3=np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test3, predict3)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test3, predict3,average='micro')
  rec_score = recall_score(y_test3, predict3, average='micro')
  f_score = f1_score(y_test3, predict3, average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)

In [ ]:
# batchsize = 32
# alexNet_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop], validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 13s 19ms/step - loss: 0.3939 - accuracy: 0.6100 - val_loss: 0.3367 - val_accuracy: 0.6767 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 8s 15ms/step - loss: 0.3439 - accuracy: 0.6688 - val_loss: 0.3363 - val_accuracy: 0.6461 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 8s 15ms/step - loss: 0.3215 - accuracy: 0.6848 - val_loss: 0.3307 - val_accuracy: 0.6384 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 9s 16ms/step - loss: 0.3104 - accuracy: 0.6948 - val_loss: 0.3006 - val_accuracy: 0.7059 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 8s 15ms/step - loss: 0.3008 - accuracy: 0.7085 - val_loss: 0.3222 - val_accuracy: 0.6845 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 8s 15ms/step - loss: 0.2954 - accuracy: 0.7076 - val_loss: 0.3170 - val_accuracy: 0.6448 - lr: 0.0010
Epoch 7/50
545/546 [============================>.] - ETA: 0s -

In [ ]:
# plt.plot(alexNet_model.history.history['accuracy'])
# # plt.plot(alexNet_model.history.history['Precision'])
# plt.plot(alexNet_model.history.history['val_accuracy'])
# # plt.plot(alexNet_model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(alexNet_model.history.history['loss'])
# plt.plot(alexNet_model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# plt.plot(alexNet_model.history.history['Recall'])
# plt.plot(alexNet_model.history.history['Precision'])
# plt.plot(alexNet_model.history.history['AUC'])
# plt.legend(("Recall","Precision","AUC"))
# plt.xlabel('Epoch')

In [ ]:
# y_pred=alexNet_model.predict(signal_test)
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test, axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# print(conf_mat)

[[899  25  42  37   1]
 [ 93 246  94 106  14]
 [ 49  14 255  35  26]
 [ 56   6   7 139   3]
 [ 38   2   3   3  10]]


In [ ]:
# accuracy_score(y_test,y_pred)

0.7031320926009986

**VGG 16**

In [ ]:
vgg_16_model=Sequential()

vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same',  input_shape=(1000,12)))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Activation('relu'))
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(GlobalAveragePooling1D())
vgg_16_model.add(Dense(256, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(128, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(5, activation='sigmoid'))

In [ ]:
#  vgg_16_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
vgg_16_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
vgg_16_model.summary()

In [ ]:
# plot_model(vgg_16_model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = vgg_16_model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = vgg_16_model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred4= vgg_16_model.predict(X_test)
  predict4 = np.argmax(y_pred4, axis=1)
  y_test4 = np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test4, predict4)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test4, predict4, average='micro')
  rec_score = recall_score(y_test4, predict4,average='micro')
  f_score = f1_score(y_test4, predict4,average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)

In [ ]:
# batchsize = 32
# vgg_16_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 42s 67ms/step - loss: 0.4073 - accuracy: 0.6016 - val_loss: 0.3824 - val_accuracy: 0.5955 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 35s 64ms/step - loss: 0.3548 - accuracy: 0.6630 - val_loss: 0.3887 - val_accuracy: 0.6215 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 35s 65ms/step - loss: 0.3291 - accuracy: 0.6911 - val_loss: 0.4888 - val_accuracy: 0.5919 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 35s 64ms/step - loss: 0.3192 - accuracy: 0.7056 - val_loss: 0.3187 - val_accuracy: 0.6708 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 35s 64ms/step - loss: 0.3077 - accuracy: 0.7123 - val_loss: 0.3593 - val_accuracy: 0.6972 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 35s 64ms/step - loss: 0.3027 - accuracy: 0.7143 - val_loss: 0.3640 - val_accuracy: 0.6530 - lr: 0.0010
Epoch 7/50
546/546 [==============================] - 35s 

In [ ]:
# plt.plot(vgg_16_model.history.history['accuracy'])
# #plt.plot(vgg_16_model.history.history['Precision'])
# plt.plot(vgg_16_model.history.history['val_accuracy'])
# #plt.plot(vgg_16_model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(vgg_16_model.history.history['loss'])
# plt.plot(vgg_16_model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# plt.plot(vgg_16_model.history.history['Recall'])
# plt.plot(vgg_16_model.history.history['Precision'])
# plt.plot(vgg_16_model.history.history['AUC'])
# plt.legend(("Recall","Precision","AUC"))
# plt.xlabel('Epoch')

In [ ]:
# y_pred=vgg_16_model.predict(signal_test)
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test,axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# print(conf_mat)

[[887  45  41  31   0]
 [ 69 307  70  91  16]
 [ 48  21 258  30  22]
 [ 46   9   8 144   4]
 [ 35   3   2   3  13]]


In [ ]:
# accuracy_score(y_test,y_pred)

0.7303676804357694

**ResNet 50 Model**

In [ ]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv1D(filters = F1, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(filters = F2, kernel_size = f, activation='relu', strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(filters = F3, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)

    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def convolutional_block(X, f, filters, s = 2):
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv1D(F1, 1, activation='relu', strides = s)(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(F2, f, activation='relu', strides = 1,padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(F3, 1, strides = 1)(X)
    X = BatchNormalization()(X)

    X_shortcut = Conv1D(F3, 1, strides = s)(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

def ResNet50(input_shape):
    
    X_input = Input(input_shape)

    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(64, 7, strides = 2)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(5,activation='sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
resNet50_model = ResNet50(input_shape = (1000,12))

In [ ]:
# resNet50_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
resNet50_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
resNet50_model.summary()

In [ ]:
# plot_model(resNet50_model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = resNet50_model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = resNet50_model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred5 = resNet50_model.predict(X_test)
  predict5 =np.argmax(y_pred5, axis=1)
  y_test5 =np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test5, predict5)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test5, predict5,average='micro')
  rec_score = recall_score(y_test5, predict5,average='micro')
  f_score = f1_score(y_test5, predict5,average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)

In [ ]:
# batchsize = 32
# resNet50_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 91s 139ms/step - loss: 0.5340 - accuracy: 0.4913 - val_loss: 0.7857 - val_accuracy: 0.5367 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 73s 133ms/step - loss: 0.3940 - accuracy: 0.6012 - val_loss: 1.1854 - val_accuracy: 0.3830 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 72s 132ms/step - loss: 0.3792 - accuracy: 0.6226 - val_loss: 0.5326 - val_accuracy: 0.6101 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 72s 132ms/step - loss: 0.3464 - accuracy: 0.6509 - val_loss: 0.4411 - val_accuracy: 0.6252 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 73s 133ms/step - loss: 0.3375 - accuracy: 0.6608 - val_loss: 0.5238 - val_accuracy: 0.5809 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 72s 133ms/step - loss: 0.3448 - accuracy: 0.6586 - val_loss: 0.3441 - val_accuracy: 0.6626 - lr: 0.0010
Epoch 7/50
546/546 [==============================] 

In [ ]:
# plt.plot(resNet50_model.history.history['accuracy'])
# #plt.plot(resNet50_model.history.history['Precision'])
# plt.plot(resNet50_model.history.history['val_accuracy'])
# #plt.plot(resNet50_model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(resNet50_model.history.history['loss'])
# plt.plot(resNet50_model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# plt.plot(resNet50_model.history.history['Recall'])
# plt.plot(resNet50_model.history.history['Precision'])
# plt.plot(resNet50_model.history.history['AUC'])
# plt.legend(("Recall","Precision","AUC"))
# plt.xlabel('Epoch')

In [ ]:
# y_pred=resNet50_model.predict(signal_test)
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test,axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# print(conf_mat)

[[883  39  39  40   3]
 [102 245  99  91  16]
 [ 52  18 254  25  30]
 [ 52  10  10 137   2]
 [ 35   1   4   3  13]]


In [ ]:
# accuracy_score(y_test,y_pred)

0.6954153427144802

**Inseption Model**

In [ ]:
def inception_block(prev_layer):
    
    conv1=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv1=BatchNormalization()(conv1)
    conv1=Activation('relu')(conv1)
    
    conv3=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
    conv3=Conv1D(filters = 64, kernel_size = 3, padding = 'same')(conv3)
    conv3=BatchNormalization()(conv3)
    conv3=Activation('relu')(conv3)
    
    conv5=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    conv5=Conv1D(filters = 64, kernel_size = 5, padding = 'same')(conv5)
    conv5=BatchNormalization()(conv5)
    conv5=Activation('relu')(conv5)
    
    pool= MaxPool1D(pool_size=3, strides=1, padding='same')(prev_layer)
    convmax=Conv1D(filters = 64, kernel_size = 1, padding = 'same')(pool)
    convmax=BatchNormalization()(convmax)
    convmax=Activation('relu')(convmax)
    
    layer_out = concatenate([conv1, conv3, conv5, convmax], axis=1)
    
    return layer_out

def inception_model(input_shape):
    X_input=Input(input_shape)
    
    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(filters = 64, kernel_size = 7, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPool1D(pool_size=3, strides=2, padding='same')(X)
    
    X = Conv1D(filters = 64, kernel_size = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = inception_block(X)
    X = inception_block(X)
    
    X = MaxPool1D(pool_size=7, strides=2, padding='same')(X)
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(5,activation='sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name='Inception')
    
    return model

In [ ]:
inception_model = inception_model(input_shape = (1000,12))

In [ ]:
# inception_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
inception_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
inception_model.summary()

In [ ]:
# plot_model(inception_model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = inception_model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = inception_model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred6= inception_model.predict(X_test)
  predict6=np.argmax(y_pred6, axis=1)
  y_test6=np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test6, predict6)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test6, predict6,average='micro')
  rec_score = recall_score(y_test6, predict6,average='micro')
  f_score = f1_score(y_test6, predict6,average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)

In [ ]:
# batchsize = 32
# inception_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 39s 64ms/step - loss: 0.3573 - accuracy: 0.6507 - val_loss: 0.3569 - val_accuracy: 0.6402 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 34s 61ms/step - loss: 0.3024 - accuracy: 0.7034 - val_loss: 0.3094 - val_accuracy: 0.7104 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 34s 61ms/step - loss: 0.2870 - accuracy: 0.7207 - val_loss: 0.3005 - val_accuracy: 0.6826 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 33s 61ms/step - loss: 0.2779 - accuracy: 0.7297 - val_loss: 0.3183 - val_accuracy: 0.7123 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 33s 61ms/step - loss: 0.2686 - accuracy: 0.7310 - val_loss: 0.2894 - val_accuracy: 0.7118 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 33s 61ms/step - loss: 0.2630 - accuracy: 0.7386 - val_loss: 0.2971 - val_accuracy: 0.6863 - lr: 0.0010
Epoch 7/50
545/546 [============================>.] - ETA:

In [ ]:
# plt.plot(inception_model.history.history['accuracy'])
# # plt.plot(inception_model.history.history['Precision'])
# plt.plot(inception_model.history.history['val_accuracy'])
# # plt.plot(inception_model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(inception_model.history.history['loss'])
# plt.plot(inception_model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# plt.plot(inception_model.history.history['Recall'])
# plt.plot(inception_model.history.history['Precision'])
# plt.plot(inception_model.history.history['AUC'])
# plt.legend(("Recall","Precision","AUC"))
# plt.xlabel('Epoch')

In [ ]:
# y_pred=inception_model.predict(signal_test)
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test,axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# print(conf_mat)

[[911  25  28  37   3]
 [ 83 283  77  90  20]
 [ 56  13 244  31  35]
 [ 51   4   7 143   6]
 [ 36   1   1   2  16]]


In [ ]:
# accuracy_score(y_test,y_pred)

0.7249205628688152

**RNN Model**

**LSTM Model**

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(1000,12), return_sequences=True))
lstm_model.add(LSTM(64))
lstm_model.add(Dense(32, activation = 'relu'))
lstm_model.add(Dropout(0.3))
lstm_model.add(Dense(5, activation = 'sigmoid'))

In [ ]:
#  lstm_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
lstm_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
lstm_model.summary()

In [ ]:
# plot_model(lstm_model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = lstm_model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = lstm_model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred7= lstm_model.predict(X_test)
  predict7=np.argmax(y_pred7, axis=1)
  y_test7 =np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test7, predict7)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test7, predict7,average='micro')
  rec_score = recall_score(y_test7, predict7,average='micro')
  f_score = f1_score(y_test7, predict7,average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)

In [ ]:
# batchsize = 32
# lstm_model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop],  validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 39s 65ms/step - loss: 0.5548 - accuracy: 0.4244 - val_loss: 0.5339 - val_accuracy: 0.4551 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 34s 62ms/step - loss: 0.5454 - accuracy: 0.4511 - val_loss: 0.5244 - val_accuracy: 0.4596 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 34s 62ms/step - loss: 0.5380 - accuracy: 0.4484 - val_loss: 0.5276 - val_accuracy: 0.4628 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 34s 61ms/step - loss: 0.5148 - accuracy: 0.4756 - val_loss: 0.5014 - val_accuracy: 0.4747 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 34s 62ms/step - loss: 0.4844 - accuracy: 0.5049 - val_loss: 0.4539 - val_accuracy: 0.5303 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 35s 63ms/step - loss: 0.4400 - accuracy: 0.5463 - val_loss: 0.4280 - val_accuracy: 0.5536 - lr: 0.0010
Epoch 7/50
546/546 [==============================] - 34s 

In [ ]:
# plt.plot(lstm_model.history.history['accuracy'])
# # plt.plot(lstm_model.history.history['Precision'])
# plt.plot(lstm_model.history.history['val_accuracy'])
# # plt.plot(lstm_model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(lstm_model.history.history['loss'])
# plt.plot(lstm_model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# plt.plot(lstm_model.history.history['Recall'])
# plt.plot(lstm_model.history.history['Precision'])
# plt.plot(lstm_model.history.history['AUC'])
# plt.legend(("Recall","Precision","AUC"))
# plt.xlabel('Epoch')

In [ ]:
# y_pred=lstm_model.predict(signal_test)
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test,axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# print(conf_mat)

[[882  45  42  32   3]
 [ 92 250 106 102   3]
 [ 61  22 249  32  15]
 [ 57  10   6 134   4]
 [ 35   0   9   2  10]]


In [ ]:
# accuracy_score(y_test,y_pred)

**CNN+LSTM**

In [ ]:
input_layer = Input(shape=(1000,12))
conv1 = Conv1D(filters=32,kernel_size=8,strides=1,activation='relu')(input_layer)
pool1 = MaxPool1D(pool_size=4)(conv1)
lstm1 = LSTM(64)(pool1)
output_layer = Dense(5, activation='sigmoid')(lstm1)
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.BinaryAccuracy(
#         name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
#                     tf.keras.metrics.AUC(
#         num_thresholds=200,
#         curve="ROC",
#         summation_method="interpolation",
#         name="AUC",
#         dtype=None,
#         thresholds=None,
#         multi_label=True,
#         label_weights=None,
#     )])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
model.summary()

In [ ]:
# plot_model(model)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 2
batchsize = 32
kfold = KFold(n_splits=n_folds, shuffle=True)
model_history = [] 

fold_no = 1
for train, test in kfold.split(X_train, Y_train):
  print("Training on Fold: ",fold_no)
  results = model.fit(X_train, Y_train, batch_size=batchsize, epochs=30, verbose='auto', callbacks=[reduce_lr,early_stop], validation_split=0.15)
  score = model.evaluate(X_test, Y_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

  y_pred8 = model.predict(X_test)
  predict8=np.argmax(y_pred8, axis=1)
  y_test8=np.argmax(Y_test, axis=1)
  conf_mat = confusion_matrix(y_test8, predict8)
  print(conf_mat)

  # acc_score=accuracy_score(Y_test,y_pred1)
  prec_score = precision_score(y_test8, predict8,average='micro')
  rec_score = recall_score(y_test8, predict8,average='micro')
  f_score = f1_score(y_test8, predict8,average='micro')
  print("Precision(test)= ", prec_score, " Recall(test)= ", rec_score, " F1 Score(test)= ", f_score)
  # fpr, tpr, thresholds = metrics.roc_curve(y_test1,y_pred1)
  # print("AUC(test)= ",metrics.auc(fpr, tpr))
  
  fold_no = fold_no + 1
  
  model_history.append(results)

In [ ]:
# batchsize = 32
# model.fit(x=signal, y=target, batch_size=batchsize, epochs=50, verbose='auto', callbacks=[reduce_lr,early_stop], validation_data=(v_signal,v_target))

Epoch 1/50
546/546 [==============================] - 9s 14ms/step - loss: 0.5122 - accuracy: 0.4888 - val_loss: 0.5314 - val_accuracy: 0.4665 - lr: 0.0010
Epoch 2/50
546/546 [==============================] - 7s 13ms/step - loss: 0.4922 - accuracy: 0.4973 - val_loss: 0.4722 - val_accuracy: 0.5125 - lr: 0.0010
Epoch 3/50
546/546 [==============================] - 7s 13ms/step - loss: 0.4604 - accuracy: 0.5279 - val_loss: 0.5081 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 4/50
546/546 [==============================] - 7s 13ms/step - loss: 0.4049 - accuracy: 0.5880 - val_loss: 0.3909 - val_accuracy: 0.5905 - lr: 0.0010
Epoch 5/50
546/546 [==============================] - 7s 13ms/step - loss: 0.3781 - accuracy: 0.6159 - val_loss: 0.3765 - val_accuracy: 0.6179 - lr: 0.0010
Epoch 6/50
546/546 [==============================] - 7s 13ms/step - loss: 0.3567 - accuracy: 0.6348 - val_loss: 0.3546 - val_accuracy: 0.6334 - lr: 0.0010
Epoch 7/50
546/546 [==============================] - 7s 12ms/st

In [ ]:
# plt.plot(model.history.history['accuracy'])
# #plt.plot(model.history.history['Precision'])
# plt.plot(model.history.history['val_accuracy'])
# #plt.plot(model.history.history['val_Precision'])
# plt.legend(("accuracy","val_accuracy"))
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')

In [ ]:
# plt.plot(model.history.history['loss'])
# plt.plot(model.history.history['val_loss'])
# plt.legend(("loss","val_loss"))
# plt.xlabel('Epoch')
# plt.ylabel('Loss')

In [ ]:
# y_pred=model.predict(signal_test)
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(target_test,axis=1)
# conf_mat=confusion_matrix(y_test,y_pred)
# print(conf_mat)

In [ ]:
# accuracy_score(y_test,y_pred)

0.6659101225601453

0.6659101225601453